In [1]:
import requests
import pandas as pd

API_KEY = "NGELwQY8b2XQcUu8sAs9a96T9COtSsacKzI0FK4gJmS0qgqGjTRWgRBfNG3sfwt0"
BASE_URL = "https://api.collegefootballdata.com/drives"

years = [2023]
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

params_list = [{'year': year, 'week': week, 'classification': 'fbs'} for year in years for week in weeks]

headers = {'Authorization': f'Bearer {API_KEY}'}

def get_data_from_api(params):
    res = requests.get(BASE_URL, params=params, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        print(f"Error fetching data for year {params['year']} week {params['week']}: {res.status_code}")
        return None

data = []
for params in params_list:
    week_data = get_data_from_api(params)
    if week_data:
        for play in week_data:
            play['week'] = params['week']
            play['year'] = params['year']
        data.extend(week_data)

df = pd.DataFrame(data)

print(df.info())
print(df.describe())


/Users/ravikurella/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20852 entries, 0 to 20851
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   offense              20852 non-null  object
 1   offense_conference   20852 non-null  object
 2   defense              20852 non-null  object
 3   defense_conference   20852 non-null  object
 4   game_id              20852 non-null  int64 
 5   id                   20852 non-null  object
 6   drive_number         20852 non-null  int64 
 7   scoring              20852 non-null  bool  
 8   start_period         20852 non-null  int64 
 9   start_yardline       20852 non-null  int64 
 10  start_yards_to_goal  20852 non-null  int64 
 11  start_time           20852 non-null  object
 12  end_period           20852 non-null  int64 
 13  end_yardline         20852 non-null  int64 
 14  end_yards_to_goal    20852 non-null  int64 
 15  end_time             20852 non-null  object
 16  elap

In [2]:
df["distance_to_goal"] = 100 - df["start_yards_to_goal"]

df["points_scored_offense"] = df["end_offense_score"] - df["start_offense_score"]
df["points_scored_defense"] = df["end_defense_score"] - df["start_defense_score"]

df["yards_gained_per_play"] = df["yards"] / df["plays"].replace(0, 1)

df['score_diff_start'] = df['start_offense_score'] - df['start_defense_score']

df['yards_gained'] = df['end_yardline'] - df['start_yardline']

df.loc[(df["end_yardline"] < 0) & (df["drive_result"] == "Uncategorized"), "end_yardline"] = 0
df.loc[(df["end_yards_to_goal"] < 0) & (df["drive_result"] == "Uncategorized"), "end_yards_to_goal"] = 0

uncategorized_issues = df[(df["end_yards_to_goal"] < 0) & (df["drive_result"] == "Uncategorized")]

df['points_scored_offense'] = df['points_scored_offense'].clip(lower=0)
df['points_scored_defense'] = df['points_scored_defense'].clip(lower=0)
print(uncategorized_issues)
print(len(df['offense'].unique()))
print(df.info())

Empty DataFrame
Columns: [offense, offense_conference, defense, defense_conference, game_id, id, drive_number, scoring, start_period, start_yardline, start_yards_to_goal, start_time, end_period, end_yardline, end_yards_to_goal, end_time, elapsed, plays, yards, drive_result, is_home_offense, start_offense_score, start_defense_score, end_offense_score, end_defense_score, week, year, distance_to_goal, points_scored_offense, points_scored_defense, yards_gained_per_play, score_diff_start, yards_gained]
Index: []

[0 rows x 33 columns]
229
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20852 entries, 0 to 20851
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   offense                20852 non-null  object 
 1   offense_conference     20852 non-null  object 
 2   defense                20852 non-null  object 
 3   defense_conference     20852 non-null  object 
 4   game_id                20852 non-n

In [3]:
import ast

def extract_time(time_str):
    try:
        time_dict = ast.literal_eval(time_str) 
        minutes = time_dict.get("minutes", 0)
        seconds = time_dict.get("seconds", 0)
        return minutes, seconds
    except:
        return 0, 0

df[['start_time_minutes', 'start_time_seconds']] = df['start_time'].apply(lambda x: pd.Series(extract_time(x)))
df[['end_time_minutes', 'end_time_seconds']] = df['end_time'].apply(lambda x: pd.Series(extract_time(x)))

df['start_time_in_seconds'] = df['start_time_minutes']*60 + df['start_time_seconds']
df['end_time_in_seconds'] = df['end_time_minutes']*60 + df['end_time_seconds']

df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20852 entries, 0 to 20851
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   offense                20852 non-null  object 
 1   offense_conference     20852 non-null  object 
 2   defense                20852 non-null  object 
 3   defense_conference     20852 non-null  object 
 4   game_id                20852 non-null  int64  
 5   id                     20852 non-null  object 
 6   drive_number           20852 non-null  int64  
 7   scoring                20852 non-null  bool   
 8   start_period           20852 non-null  int64  
 9   start_yardline         20852 non-null  int64  
 10  start_yards_to_goal    20852 non-null  int64  
 11  start_time             20852 non-null  object 
 12  end_period             20852 non-null  int64  
 13  end_yardline           20852 non-null  int64  
 14  end_yards_to_goal      20852 non-null  int64  
 15  en

(None,
               offense offense_conference             defense  \
 0  Jacksonville State     Conference USA                UTEP   
 1                UTEP     Conference USA  Jacksonville State   
 2  Jacksonville State     Conference USA                UTEP   
 3                UTEP     Conference USA  Jacksonville State   
 4  Jacksonville State     Conference USA                UTEP   
 
   defense_conference    game_id          id  drive_number  scoring  \
 0     Conference USA  401520145  4015201451             1    False   
 1     Conference USA  401520145  4015201452             2    False   
 2     Conference USA  401520145  4015201453             3     True   
 3     Conference USA  401520145  4015201454             4    False   
 4     Conference USA  401520145  4015201455             5     True   
 
    start_period  start_yardline  ...  points_scored_defense  \
 0             1              25  ...                      0   
 1             1              95  ...        

In [4]:
from sklearn.preprocessing import LabelEncoder

# Label encoding offense-defense matchups
matchup_encoder = LabelEncoder()
df["offense_defense_matchup_encoded"] = matchup_encoder.fit_transform(df["offense"] + "_" + df["defense"])
df['offense_idx'] = matchup_encoder.fit_transform(df['offense'])
df['defense_idx'] = matchup_encoder.fit_transform(df['defense'])
df['offense_conference_idx'] = matchup_encoder.fit_transform(df['offense_conference'])
df['defense_conference_idx'] = matchup_encoder.fit_transform(df['defense_conference'])
# Compute average points scored per matchup
df["offense_defense_matchup"] = df["offense"] + "_" + df["defense"]
matchup_avg_points_by_offense = df.groupby("offense_defense_matchup")["points_scored_offense"].mean()
matchup_avg_points_by_defense = df.groupby("offense_defense_matchup")["points_scored_defense"].mean()
df["offense_defense_matchup_avg_points_by_offense_points"] = df["offense_defense_matchup"].map(matchup_avg_points_by_offense)
df["offense_defense_matchup_avg_points_by_defense_points"] = df["offense_defense_matchup"].map(matchup_avg_points_by_defense)

matchup_avg_yards = df.groupby('offense_defense_matchup')['yards_gained'].mean()
df['matchup_avg_yards'] = df['offense_defense_matchup'].map(matchup_avg_yards)
df['is_home_offense'] = df['is_home_offense'].astype(int)
df.info()

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20852 entries, 0 to 20851
Data columns (total 48 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   offense                                               20852 non-null  object 
 1   offense_conference                                    20852 non-null  object 
 2   defense                                               20852 non-null  object 
 3   defense_conference                                    20852 non-null  object 
 4   game_id                                               20852 non-null  int64  
 5   id                                                    20852 non-null  object 
 6   drive_number                                          20852 non-null  int64  
 7   scoring                                               20852 non-null  bool   
 8   start_period                                          20

,offense,offense_conference,defense,defense_conference,game_id,id,drive_number,scoring,start_period,start_yardline,...,end_time_in_seconds,offense_defense_matchup_encoded,offense_idx,defense_idx,offense_conference_idx,defense_conference_idx,offense_defense_matchup,offense_defense_matchup_avg_points_by_offense_points,offense_defense_matchup_avg_points_by_defense_points,matchup_avg_yards
0,Jacksonville State,Conference USA,UTEP,Conference USA,401520145,4015201451,1,False,1,25,...,0,581,80,204,7,7,Jacksonville State_UTEP,1.545455,0.000000,27.272727
1,UTEP,Conference USA,Jacksonville State,Conference USA,401520145,4015201452,2,False,1,95,...,0,1542,204,80,7,7,UTEP_Jacksonville State,1.272727,0.000000,-36.636364
2,Jacksonville State,Conference USA,UTEP,Conference USA,401520145,4015201453,3,True,1,53,...,0,581,80,204,7,7,Jacksonville State_UTEP,1.545455,0.000000,27.272727
3,UTEP,Conference USA,Jacksonville State,Conference USA,401520145,4015201454,4,False,1,75,...,0,1542,204,80,7,7,UTEP_Jacksonville State,1.272727,0.000000,-36.636364
4,Jacksonville State,Conference USA,UTEP,Conference USA,401520145,4015201455,5,True,1,20,...,0,581,80,204,7,7,Jacksonville State_UTEP,1.545455,0.000000,27.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20847,Army,FBS Independents,Navy,American Athletic,401520445,40152044517,17,False,4,53,...,0,110,12,116,8,1,Army_Navy,0.000000,1.000000,-46.100000
20848,Navy,American Athletic,Army,FBS Independents,401520445,40152044518,18,True,4,30,...,0,876,116,12,1,8,Navy_Army,0.545455,0.818182,-18.909091
20849,Navy,American Athletic,Army,FBS Independents,401520445,40152044519,19,True,4,0,...,0,876,116,12,1,8,Navy_Army,0.545455,0.818182,-18.909091
20850,Army,FBS Independents,Navy,American Athletic,401520445,40152044520,20,False,4,0,...,0,110,12,116,8,1,Army_Navy,0.000000,1.000000,-46.100000


In [ ]:
df['goal_to_go'] = (df['start_yards_to_goal'] <= 10).astype(int)
df['is_off_adv'] = (df['start_offense_score'] > df['start_defense_score']).astype(int)
df['is_def_adv'] = (df['start_defense_score'] > df['start_offense_score']).astype(int)

df['score_diff_start_off_adv'] = ((df['start_offense_score'] - df['start_defense_score'])>0).astype(int)
df['score_diff_start_def_adv'] = ((df['start_defense_score'] - df['start_offense_score'])>0).astype(int)
df.to_csv('../data/drives2023.csv', index=False)
